In [1]:
import pandas as pd

file_path = 'epn_cleaned.csv'

df = pd.read_csv(file_path)

print(df.head().to_markdown())

|    | FIRSTNAME    | LASTNAME                |      BIN | MOBILE         | EMAIL                      |   PRIMARYCARD | CARD_CREATION_DATE   |   ACCOUNTSERNO |   CARDSERNO |   RMNAME |
|---:|:-------------|:------------------------|---------:|:---------------|:---------------------------|--------------:|:---------------------|---------------:|------------:|---------:|
|  0 | unidentified | ANSH                    | 42113700 | (91)9398742700 | anshsinghvee@gmail.com     |             0 | 10Jul2023            |       27965111 |    37346654 |      nan |
|  1 | unidentified | ANUSH                   | 42113700 | (91)9626999111 | aswin@herofashion.com      |             0 | 26Nov2021            |       29337588 |    20615046 |      nan |
|  2 | unidentified | GOUTHAM                 | 42113700 | (91)9008839300 | samarath@prithvijewels.com |             0 | 12Jul2022            |       39259747 |    26753794 |      nan |
|  3 | unidentified | LAKSHMI PRABHA SELVARAJ | 42113700 | (91)9003711

In [8]:
!pip install email_validator



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for email_validator from https://files.pythonhosted.org/packages/09/68/d237a603d524ba052e292d71c89939bfa70e3ec7963b255cc3ef7a8770a0/email_validator-2.1.0.post1-py3-none-any.whl.metadata
  Obtaining dependency information for dnspython>=2.0.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/300.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.4 kB ? eta -

In [3]:
df_copy = df.copy()

business_domains = ['hotmail.com', 'gmail.com', 'outlook.com'] 
df_copy['is_business'] = df_copy['EMAIL'].apply(lambda x: 0 if any(domain in x.lower() for domain in business_domains) else 1)

print(df_copy.head().to_markdown())

|    | FIRSTNAME    | LASTNAME                |      BIN | MOBILE         | EMAIL                      |   PRIMARYCARD | CARD_CREATION_DATE   |   ACCOUNTSERNO |   CARDSERNO |   RMNAME |   is_business |
|---:|:-------------|:------------------------|---------:|:---------------|:---------------------------|--------------:|:---------------------|---------------:|------------:|---------:|--------------:|
|  0 | unidentified | ANSH                    | 42113700 | (91)9398742700 | anshsinghvee@gmail.com     |             0 | 10Jul2023            |       27965111 |    37346654 |      nan |             0 |
|  1 | unidentified | ANUSH                   | 42113700 | (91)9626999111 | aswin@herofashion.com      |             0 | 26Nov2021            |       29337588 |    20615046 |      nan |             1 |
|  2 | unidentified | GOUTHAM                 | 42113700 | (91)9008839300 | samarath@prithvijewels.com |             0 | 12Jul2022            |       39259747 |    26753794 |      nan |       

In [4]:
df_copy['nri'] = df_copy['MOBILE'].apply(lambda x: 1 if x[:2] != '91' else 0)

print(df_copy.head().to_markdown())

|    | FIRSTNAME    | LASTNAME                |      BIN | MOBILE         | EMAIL                      |   PRIMARYCARD | CARD_CREATION_DATE   |   ACCOUNTSERNO |   CARDSERNO |   RMNAME |   is_business |   nri |
|---:|:-------------|:------------------------|---------:|:---------------|:---------------------------|--------------:|:---------------------|---------------:|------------:|---------:|--------------:|------:|
|  0 | unidentified | ANSH                    | 42113700 | (91)9398742700 | anshsinghvee@gmail.com     |             0 | 10Jul2023            |       27965111 |    37346654 |      nan |             0 |     1 |
|  1 | unidentified | ANUSH                   | 42113700 | (91)9626999111 | aswin@herofashion.com      |             0 | 26Nov2021            |       29337588 |    20615046 |      nan |             1 |     1 |
|  2 | unidentified | GOUTHAM                 | 42113700 | (91)9008839300 | samarath@prithvijewels.com |             0 | 12Jul2022            |       39259747 |

In [12]:
from email_validator import validate_email

try:
    df_copy['dead_mail'] = df_copy['EMAIL'].apply(lambda x: 1 if not validate_email(x) else 0)
    df_copy['not_deliverable'] = df_copy['EMAIL'].apply(lambda x: 1 if validate_email(x, check_deliverability=True) == False else 0)
except Exception as e:
    print(f"An error occurred: {e}")

print(df_copy.head().to_markdown())


An error occurred: The domain name apeamysore.com does not exist.
|    | FIRSTNAME    | LASTNAME                |      BIN | MOBILE         | EMAIL                      |   PRIMARYCARD | CARD_CREATION_DATE   |   ACCOUNTSERNO |   CARDSERNO |   RMNAME |   is_business |   nri |
|---:|:-------------|:------------------------|---------:|:---------------|:---------------------------|--------------:|:---------------------|---------------:|------------:|---------:|--------------:|------:|
|  0 | unidentified | ANSH                    | 42113700 | (91)9398742700 | anshsinghvee@gmail.com     |             0 | 10Jul2023            |       27965111 |    37346654 |      nan |             0 |     1 |
|  1 | unidentified | ANUSH                   | 42113700 | (91)9626999111 | aswin@herofashion.com      |             0 | 26Nov2021            |       29337588 |    20615046 |      nan |             1 |     1 |
|  2 | unidentified | GOUTHAM                 | 42113700 | (91)9008839300 | samarath@prithvije